In [1]:
import folium
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D
from tensorflow import expand_dims
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import r2_score

The following is some leftover code from when I tried to make a neural network based on the peak 30 seconds of activity across sensors. As with other cases, many NN structures were tried and the one model below is not indicative of everything I did (they just never did much other than converge to the mean....in a good run).

In [2]:
wf_df=pd.read_pickle("..\..\data\pickled_files\\wf_df.p")

In [3]:
device_cols=wf_df.columns[wf_df.columns.str.startswith('device')]

In [4]:
X = wf_df[device_cols]
y= wf_df[['latitude', 'longitude']]

In [5]:
def make_arrays(df_slice):
    convert = np.array(df_slice)
    output=[]
    i=0
    for row in convert:
        array = np.dstack(row)
        output.append(array)
        i+=1
    return np.array(output)

In [6]:
X_arr= make_arrays(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_arr, y, random_state=69, test_size=0.2, stratify=wf_df['outlier'])

In [8]:
model=Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,2), activation='relu', padding='same', input_shape=(3,30,28)))
model.add(Flatten())
model.add(Dense(1024, kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='linear'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 30, 32)         5408      
_________________________________________________________________
flatten (Flatten)            (None, 2880)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2950144   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2050      
Total params: 2,957,602
Trainable params: 2,957,602
Non-trainable params: 0
_________________________________________________________________


In [9]:
opt=optimizers.Adam(learning_rate=0.001)
model.compile(loss = 'mse', optimizer=opt, metrics=['mse'])

In [17]:
early_stop=EarlyStopping(monitor='val_mse', patience=10, min_delta=0.5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_mse', factor=0.5, patience=10)
loc_history=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=8, callbacks=[reduce_lr, early_stop])

Epoch 1/100
100/100 [==============================] - 1s 6ms/step - loss: 152.3507 - mse: 79.3121 - val_loss: 101.5781 - val_mse: 29.4403
Epoch 2/100
100/100 [==============================] - 0s 5ms/step - loss: 137.9750 - mse: 66.6964 - val_loss: 99.6004 - val_mse: 29.1608
Epoch 3/100
100/100 [==============================] - 0s 5ms/step - loss: 142.7623 - mse: 73.0896 - val_loss: 97.6143 - val_mse: 28.7144
Epoch 4/100
100/100 [==============================] - 1s 5ms/step - loss: 127.5824 - mse: 59.3898 - val_loss: 95.5852 - val_mse: 28.0543
Epoch 5/100
100/100 [==============================] - 0s 5ms/step - loss: 132.0654 - mse: 65.2290 - val_loss: 91.8886 - val_mse: 25.7767
Epoch 6/100
100/100 [==============================] - 0s 5ms/step - loss: 123.3279 - mse: 57.8756 - val_loss: 91.1494 - val_mse: 26.3612
Epoch 7/100
100/100 [==============================] - 0s 5ms/step - loss: 127.6698 - mse: 63.5167 - val_loss: 89.6826 - val_mse: 26.1769
Epoch 8/100
100/100 [============

In [14]:
train_preds=model.predict(X_train)
test_preds=model.predict(X_test)

In [15]:
r2_score(y_train, train_preds)

-0.2957766394289585

In [16]:
r2_score(y_test, test_preds)

-0.3716878038747423